In [ ]:
import logging, json
import numpy as np
import pandas as pd
import time

from apsopt.core.optimizer import Optimizer, get_generator_class
from apsopt.core.evaluators import FunctionEvaluator
from apsopt.util.pydantic.options import AlgorithmOptions
from apsopt.util.pydantic.vocs import GVOCS

from opt_funcs_nsls import make_dummy_eval, make_dummy_eval_lifetime

logger = logging.getLogger(__name__)

In [ ]:
MODE = '6and3'
with open(f'lattice/data_{MODE}.json', 'r') as f:
    data_dict = json.load(f)
groups = data_dict['groups']
groups_direct = data_dict['groups_direct']
all_groups = {**groups, **groups_direct}
indices = data_dict['indices']
null_knobs = data_dict['null_knobs']
dfknobs = pd.read_json(f'lattice/dfknobs_{MODE}.json')

In [ ]:
# impact of K2L difference
dfknobs.loc['SM2B',:] *= 0.25/0.2

In [ ]:
dfknobs

This notebook uses:
- Absolute K2L offsets per knob as variables
- Absolute K2L values per family as inputs to the evaluation function
- Lifetime as the objective

SETUP SINGLE-OBJECTIVE RUN (i.e. only lifetime)

In [ ]:
# DO NOT CHANGE - USED TO CALCULATE ABSOLUTE K2L
initial_values_ref = {
    'SM1A': -23.68063424*0.2,
    'SM2B': 28.64315469*0.25,
    'SM1B': -25.94603546*0.2,
    'SL3': -29.46086061*0.2,
    'SL2': 35.67792145*0.2,
    'SL1': -13.27160605*0.2,
    'SH4': -15.82090071*0.2,
    'SH3': -5.85510841*0.2,
    'SH1': 19.8329121*0.2,
}

# CHANGE THIS TO VALUES FROM RING IF NEEDED
initial_values = initial_values_ref.copy()

# optimizer variables in units of K2L
variables = {'SVD0':[-0.6,0.6]} # this means SM1B will change by ~10%
             
for k in ['SH1','SH3','SH4','SL1','SL2','SL3']:
    variables[k] = [-np.abs(initial_values_ref[k])*0.1,np.abs(initial_values_ref[k])*0.1]

objectives_active = {'LT':'MAXIMIZE'}

initial_variable_values = {k:0.0 for k in variables.keys()}

gvocs = GVOCS(variables=variables,
              variables_active=variables,
              objectives=objectives_active,
              objectives_active=objectives_active)

print(pd.DataFrame(initial_values_ref,index=[0]))
print(pd.DataFrame(variables))

In [ ]:
import sext_tools
def compute_family_k2l_from_knob_k2l(dfknobs, knobs_dict, debug=False):
    group_relative_strengths = sext_tools.knob_strengths_to_group_strengths(dfknobs, knobs_dict)
    for k in group_relative_strengths:
        if debug:
            print(f'{k}: {initial_values[k]:.3f} + {group_relative_strengths[k]:.3f} -> {initial_values[k]+group_relative_strengths[k]:.3f}')
        group_relative_strengths[k] = group_relative_strengths[k] + initial_values[k]        
    return group_relative_strengths

In [ ]:
compute_family_k2l_from_knob_k2l(dfknobs, {k:variables[k][0] for k in variables.keys()}, True)

In [ ]:
eval_f = make_dummy_eval_lifetime(gvocs)
ev = FunctionEvaluator(script=eval_f)

generator_name = 'bo_ucb'
gc = get_generator_class(generator_name)

gen = gc(gvocs=gvocs,
         turbo_controller='optimize',
        )
gen.numerical_optimizer.n_raw_samples = 512
gen.numerical_optimizer.n_restarts = 10
gen.gp_constructor.use_low_noise_prior = False

In [ ]:
options = AlgorithmOptions(store_history=True, dump_every_step=True)
opt = Optimizer(options=options,
                generator=gen,
                evaluator=ev,
                gvocs=gvocs)

Sample at initial values (i.e. standard lattice)

In [ ]:
opt.generate_at(initial_variable_values)
opt.evaluate_data()

In [ ]:
opt.data

In [ ]:
opt.state_to_json()

Sample close to initial point

In [ ]:
mt = {k: 0.05 for k in variables.keys()}
opt.generate_random_around_point(n=1, point=initial_variable_values, spans=mt, use_normalized_distances=True)
opt.evaluate_data()

In [ ]:
opt.data

Run optimizer

In [ ]:
import warnings
#warnings.filterwarnings("ignore", message="Unable to find non-zero acquisition function values")

for i in range(50):
    opt.step()

In [ ]:
opt.data.tail()

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,2,squeeze=False,figsize=(12,6))
df = opt.data
ol = ['LT']

ax[0,1].plot(df.loc[:,gvocs.objective_names_active[0]],marker='o')
ax[0,1].set_xlabel('Step #')
ax[0,1].set_ylabel('Objective')

In [ ]:
# This will save optimizer state
dt = time.strftime("%Y_%m_%d-%H_%M_%S")
opt.state_to_json(f'sobo_dump_{dt}.json')